<a href="https://colab.research.google.com/github/barryhpr/SyntheticDebiasing/blob/main/Adapter_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.26.1
!pip install -U adapter-transformers
!pip install datasets
!git clone https://github.com/barryhpr/SyntheticDebiasing.git
%cd /content/SyntheticDebiasing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
Cloning into 'SyntheticDebiasing'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (34/34), 

In [ ]:
#@title Original Score for GPT2
from Score_evaluator import *
from Score_evaluator1_1 import *
from Evaluate_StereoSet import *
from Evaluate_CrowSPairs import *
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


#Input file: "test.json" to evaluate all categories. For a specific category, use for example: "test_gender.json".
srunner = StereoSetRunner(
    intrasentence_model = model,
    tokenizer = tokenizer,
    # model_name = "gpt2",
    input_file="test.json",
    is_generative=True,
    is_self_debias=False
    )

metric_data = srunner() # a nested dict

import json
with open('stereoset_result.json', 'w') as outfile:
    json.dump(metric_data, outfile, indent=4)

parse_file("test.json", "stereoset_result.json")
# parse_file1("test.json","stereoset_result.json", "gender")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Using mask_token, but it is not set yet.


Evaluating intrasentence task.


100%|██████████| 6392/6392 [07:06<00:00, 14.99it/s]


intrasentence
	gender
		Count: 2313.0
		LM Score: 92.01218436319637
		SS Score: 62.6460866303445
		ICAT Score: 68.74030327311215
	profession
		Count: 7194.0
		LM Score: 90.74056951271717
		SS Score: 61.305512436683145
		ICAT Score: 70.22319676996246
	race
		Count: 8928.0
		LM Score: 90.9524178709514
		SS Score: 58.904167347320374
		ICAT Score: 74.75530688362413
	religion
		Count: 741.0
		LM Score: 91.20561944224038
		SS Score: 63.260092672402635
		ICAT Score: 67.01772012128048
	overall
		Count: 6392.0
		LM Score: 91.01442389024014
		SS Score: 60.42310108475473
		ICAT Score: 72.04137308266637
overall
	Count: 6392.0
	LM Score: 91.01442389024014
	SS Score: 60.42310108475473
	ICAT Score: 72.04137308266637


In [ ]:
#@title Train GPT2
import json
import shutil
import os
import random
import torch
from torch.utils.data import Dataset
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from Evaluate_CrowSPairs import CrowSPairsRunner

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

class MyDataset(Dataset):
    def __init__(self, filename, tokenizer):
        with open(filename, 'r') as file:
            self.data = json.load(file)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = f"{item['prompt'].replace('{}', item['subject'])} {item['target_new']['str']}"
        encoding = self.tokenizer(text, return_tensors='pt')
        labels = encoding['input_ids'].clone()
        labels[:, :-1] = -100
        return {'input_ids': encoding['input_ids'].squeeze(), 'attention_mask': encoding['attention_mask'].squeeze(), 'labels': labels.squeeze()}

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#Datasets are like "Synthetic_Religion_3.json", remember don't use S. So for exmaple, don't use "Synthetic_Religion_3S"
train_dataset = MyDataset('Synthetic_Gender_2.json', tokenizer)
test_dataset = MyDataset('test.json', tokenizer)

if os.path.exists('./results'):
    shutil.rmtree('./results')
os.makedirs('./results')


# Training session
training_epoch = 5
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.add_adapter("debias")
model.train_adapter("debias")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=training_epoch,
    per_device_train_batch_size=1,
    logging_steps=10,
    seed=42,
    learning_rate=5e-6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

***** Running training *****
  Num examples = 500
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 894528


Step,Training Loss
10,11.958600
20,10.460300
30,9.184200
40,10.872800
50,10.476600
60,11.730800
70,12.389100
80,10.397100
90,11.163700
100,11.449100


KeyboardInterrupt: ignored

In [ ]:
#@title Evaluate GPT2 Model on StereoSet & CrowSPairs
# Evaluation session
from Score_evaluator import *
from Score_evaluator1_1 import *
from Evaluate_StereoSet import *
from Evaluate_CrowSPairs import *
evaluation_list = [1,2,3]
for eval_epoch in evaluation_list:

    print(f"Evaluate epoch{eval_epoch}")
    #this should be 500 for normal training
    checkpoint_path = f"/content/SyntheticDebiasing/results/checkpoint-{eval_epoch*1000}"
    eval_model = GPT2LMHeadModel.from_pretrained(checkpoint_path)
    eval_model.set_active_adapters("debias")

    ##########################################################

# please import input_file eg. "test-religion.json" for evaluating religion bias

    srunner = StereoSetRunner(
        intrasentence_model = eval_model,
        tokenizer = tokenizer,
        input_file="test-race.json",
        is_generative=True,
        is_self_debias=False
        )

    metric_data = srunner() # a nested dict

    import json
    with open('stereoset_result.json', 'w') as outfile:
        json.dump(metric_data, outfile, indent=4)  # The `indent=4` makes the JSON output more readable

    parse_file1("test.json",
		'stereoset_result.json', "race")


    #########################################################
    runner = CrowSPairsRunner(
        model=eval_model,
        tokenizer=tokenizer,
        input_file="crows_pairs_anonymized.csv",
        bias_type="race",
        is_generative=True
    )

    results = runner()
    print(f"Metric Score: {results}")
    print(" ")

In [ ]:
#@title Evaluate GPT2 Model on BiasTestGPT

import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

## Applicable datasets names ###

# Formatted_Profession_vs_Gender
# Formatted_Math_Arts_vs_Male_Female
# Formatted_Mexican_Female_European_Male_vs_MFemergent_EMintersectional
# Formatted_YoungName_OldName_vs_Pleasant_Unpleasant

# Load the data from the file (you can change datasets names here)
with open('Formatted_Profession_vs_Gender.json', 'r') as file:
    data = json.load(file)

def get_sentence_probability(model, sentence):
    """Get probability of a sentence using the GPT-2 model."""
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    log_likelihood = outputs[0].item()
    return log_likelihood

evaluation_list = [1,2,3,5]

for eval_epoch in evaluation_list:
    print(f"Evaluate epoch {eval_epoch}")

    # This should be 500 for normal training
    checkpoint_path = f"/content/SyntheticDebiasing/results/checkpoint-{eval_epoch*500}"
    eval_model = GPT2LMHeadModel.from_pretrained(checkpoint_path)
    eval_model.set_active_adapters("debias")

    # Initialize counters
    stereotype_count = 0
    total_count = len(data)

    # Iterate over each entry
    for entry in data:
        sentence_prob = get_sentence_probability(eval_model, entry["sentence"])
        alt_sentence_prob = get_sentence_probability(eval_model, entry["alt_sentence"])

        if entry["label_1"] == "stereotype":
            if sentence_prob < alt_sentence_prob:
                stereotype_count += 1
        elif entry["label_2"] == "stereotype":
            if alt_sentence_prob < sentence_prob:
                stereotype_count += 1

    # Calculate the stereotype score as a percentage
    stereotype_score = (stereotype_count / total_count) * 100
    print(f"Stereotype Score for epoch {eval_epoch}: {stereotype_score:.2f}%")



loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12

Evaluate epoch 1


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/SyntheticDebiasing/results/checkpoint-500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file /content/SyntheticDebiasing/results/checkpoint-500/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

loading configuration file /content/SyntheticDebiasing/results/checkpoint-1000/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "adapters": {
    "adapters": {
      "debias": "pfeiffer"
    },
    "con

Stereotype Score for epoch 1: 65.62%
Evaluate epoch 2


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/SyntheticDebiasing/results/checkpoint-1000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file /content/SyntheticDebiasing/results/checkpoint-1000/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

loading configuration file /content/SyntheticDebiasing/results/checkpoint-1500/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "adapters": {
    "adapters": {
      "debias": "pfeiffer"
    },
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn

Stereotype Score for epoch 2: 64.25%
Evaluate epoch 3


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/SyntheticDebiasing/results/checkpoint-1500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file /content/SyntheticDebiasing/results/checkpoint-1500/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

loading configuration file /content/SyntheticDebiasing/results/checkpoint-2500/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "adapters": {
    "adapters": {
      "debias": "pfeiffer"
    },
    "c

Stereotype Score for epoch 3: 64.12%
Evaluate epoch 5


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/SyntheticDebiasing/results/checkpoint-2500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file /content/SyntheticDebiasing/results/checkpoint-2500/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



Stereotype Score for epoch 5: 63.50%
